In [4]:
import numpy as np
import pandas as pd
#import dask.dataframe as dd

N, f = 1_000, 30

X = np.random.rand(N, f)
X[:, 0] = (X[:, 0]*3).astype(int)
X[:, 1] = (X[:, 1]*9).astype(int)
X[:, 2] = (X[:, 2]*50).astype(int)
X[:, 3] = (X[:, 3]*100).astype(int)
y = (np.random.rand(N,)*2).astype(int)

df = pd.DataFrame(X)
df[0] = df[0].astype('category')
df[1] = df[1].astype('category')
df[2] = df[2].astype('category')
df[3] = df[3].astype('category')

In [12]:
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('ohe', ce.OneHotEncoder(cols=[0, 1], impute_missing=False)),
    ('te', ce.TargetEncoder(cols=[2,3], impute_missing=False)),
    ('rfc', RandomForestClassifier())])

grid = {'rfc__n_estimators': np.arange(1,20) }

In [13]:
pipeline.fit(X, y)

Pipeline(memory=None,
     steps=[('ohe', OneHotEncoder(cols=[0, 1], drop_invariant=False, handle_unknown='impute',
       impute_missing=False, return_df=True, use_cat_names=False,
       verbose=0)), ('te', TargetEncoder(cols=[2, 3], drop_invariant=False, handle_unknown='impute',
       impute_missing=False, min_samples_lea...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [14]:
from dask_searchcv import RandomizedSearchCV, GridSearchCV
import sklearn.model_selection as ms

sklearn_grid_search = ms.GridSearchCV(pipeline, grid, n_jobs=-1)
dask_grid_search = GridSearchCV(pipeline, grid, n_jobs=-1)

In [7]:
%%timeit -n 1 -r 5
dask_grid_search.fit(X, y)

1min 53s ± 10.4 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [16]:
%%timeit -n 1 -r 5
sklearn_grid_search.fit(X, y)

33.1 s ± 1.63 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [ ]:
# Isolate that preprocessing is what takes a lot of time.

In [6]:
%%timeit -n 10 -r 3
ce.OneHotEncoder(cols=[0, 1], impute_missing=False).fit_transform(X)

28.8 ms ± 1.44 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [7]:
X2= ce.OneHotEncoder(cols=[0, 1], impute_missing=False).fit_transform(X)

In [9]:
%%timeit -n 10 -r 3
ce.TargetEncoder(cols=[2, 3], impute_missing=False).fit_transform(X2, y)

857 ms ± 18.5 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [10]:
X3 = ce.TargetEncoder(cols=[2, 3], impute_missing=False).fit_transform(X2, y)

In [17]:
%%timeit -n 3 -r 3
RandomForestClassifier(n_estimators=20).fit(X3, y)

74.8 ms ± 3.15 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [19]:
%%timeit -n 1 -r 5
pipeline.fit(X, y)

893 ms ± 19.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
